In [ ]:
# adapted from https://nn.labml.ai/diffusion/ddpm/index.html



# Overview


1. implementing the forward process 
    a. designing a noise schedule and,
    b. noising samples from the training data
2. implementing the loss
3. sampling from the model

# Setup

# Exercises

#

The forward process adds noise to the data $x_0 \sim q(x_0)$, for $T$ timesteps.

\begin{align}
q(x_t | x_{t-1}) = \mathcal{N}\big(x_t; \sqrt{1-  \beta_t} x_{t-1}, \beta_t \mathbf{I}\big) \\
q(x_{1:T} | x_0) = \prod_{t = 1}^{T} q(x_t | x_{t-1})
\end{align}

where $\beta_1, \dots, \beta_T$ is the variance schedule.

We can sample $x_t$ at any timestep $t$ with,

\begin{align}
q(x_t|x_0) &= \mathcal{N} \Big(x_t; \sqrt{\bar\alpha_t} x_0, (1-\bar\alpha_t) \mathbf{I} \Big)
\end{align}

where $\alpha_t = 1 - \beta_t$ and $\bar\alpha_t = \prod_{s=1}^t \alpha_s$

## Exercise 1a: implement the noise schedule

First, we need to compute alpha and beta for all timesteps.

### Plot the noise overtime

## Exercise 1b: sample from the forward process (noise the images)

### Plot the noised samples over time

## Exercise 2: implement the loss

## Exercise 3: generate samples (backwards process)

## Exercise 4: visualize sampling

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt


def visualize_sampling(X_noise):
    interval = 5
    every_nth_image = 2
    f = plt.figure()
    plt.axis("off")
    im = plt.imshow(X_noise[0][0, 0].cpu().detach(), cmap="gray", vmin=0, vmax=1)

    def update(i):
        raise NotImplementedError(
            "This function is not implemented. Please implement the update function to visualize the images."
        )

    anim = animation.FuncAnimation(
        f,
        update,
        frames=np.arange(0, int(len(X_noise) * 1.2), every_nth_image),
        interval=1,
        repeat=True,
        blit=True,
    )
    anim.save("generation.gif", writer="pillow")


HTML('<img src="generation.gif" width="400">')